In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim

from torch.distributions.log_normal import LogNormal

# Internal packages
from data_loader import KidneyStoneDataset, ToTensor
from model import binary_ks_net, binary_neg_loglik, cont_rec_ks_net, cont_rec_neg_loglik, cont_size_ks_net, cont_size_neg_loglik
from train import train

## Log-Normal parametrization

In [2]:
# Hyperparameters
BATCH_SIZE = 128
EPOCHS     = 150
LEARN_R    = 1e-2
N_HU       = 5

# Initialize the dataset
data = KidneyStoneDataset("./data/ks_non_linear_data.npy", transform=ToTensor())
train_loader = DataLoader(data, batch_size=BATCH_SIZE)

# Initialize the model
model = cont_size_ks_net(N_HU)

# Optimizers
#optimizer = optim.SGD(model.parameters(), lr=LEARN_R, weight_decay=0.1)
optimizer = optim.RMSprop(model.parameters(), lr=LEARN_R)

In [3]:
cum_loss = train(model, optimizer, cont_size_neg_loglik, train_loader, EPOCHS)

In [4]:
# First, we get the parameters of the size variable:
n_samples = 100000
arbitrary_query = torch.tensor([1., 1., 1.]) # It is only important that the first element is 1

# Extract the parameters
mu_L, log_sigma_L, _, _, _ = model(arbitrary_query.unsqueeze(0))
sigma_L = torch.exp(log_sigma_L)

# Sample from the distribution
L_dist = LogNormal(mu_L, sigma_L)
L_samples = L_dist.sample((n_samples,)).view(n_samples,1)
L_prob = torch.exp(L_dist.log_prob(L_samples))

In [5]:
# "Create" the queries using the samples from the distribution
t_1 = torch.cat((L_samples, torch.ones(n_samples, 2)), 1)
t_0 = torch.cat((L_samples, torch.zeros(n_samples, 1) ,torch.ones(n_samples, 1)), 1)

In [6]:
extreme_1 = torch.cat((torch.tensor(10000000000).view(-1,1).float(), torch.ones(1, 2)), 1)
extreme_0 = torch.cat((torch.tensor(10000000000).view(-1,1).float(), torch.zeros(1, 1) ,torch.ones(1, 1)), 1)
# Probabilities with Treatment A
_, _, _, mu_t1, log_sigma_t1 = model(extreme_1)
_, _, _, mu_t0, log_sigma_t0 = model(extreme_0)
print(mu_t1)

tensor([[8.6337e+09]], grad_fn=<AddmmBackward>)


In [7]:
model(extreme_1)

(tensor([[2.3429]], grad_fn=<MmBackward>),
 tensor([[-1.2496]], grad_fn=<MmBackward>),
 tensor([[1.]], grad_fn=<SigmoidBackward>),
 tensor([[8.6337e+09]], grad_fn=<AddmmBackward>),
 tensor([[3.7118e+08]], grad_fn=<AddmmBackward>))

In [8]:
model(extreme_0)

(tensor([[2.3429]], grad_fn=<MmBackward>),
 tensor([[-1.2496]], grad_fn=<MmBackward>),
 tensor([[1.]], grad_fn=<SigmoidBackward>),
 tensor([[8.6337e+09]], grad_fn=<AddmmBackward>),
 tensor([[3.7118e+08]], grad_fn=<AddmmBackward>))

In [12]:
mu_t0.item()

8633681920.0

In [13]:
mu_t1.item()

8633681920.0

In [14]:
# Probabilities with Treatment A
_, _, _, mu_t1, log_sigma_t1 = model(t_1)
_, _, _, mu_t0, log_sigma_t0 = model(t_0)

In [15]:
mu_t1_less_10 = mu_t1[L_samples < 10]
mu_t1_gret_10 = mu_t1[L_samples >= 10]
mu_t0_less_10 = mu_t0[L_samples < 10]
mu_t0_gret_10 = mu_t0[L_samples >= 10]

In [17]:
for n in [1, 5, 25, 50, 200, 2000, 20000]:
    print("%d: %f" % (n, torch.mean(mu_t1_gret_10[:n])-torch.mean(mu_t0_gret_10[:n])))

1: 10.737598
5: 10.552355
25: 10.623676
50: 10.571201
200: 10.618073
2000: 10.613517
20000: 10.621495


In [17]:
mu_t1-mu_t0

tensor([[5.7287]], grad_fn=<SubBackward0>)

In [24]:
# By monte carlo integration, we are getting mu from the value of the variables rom the distribution
torch.mean(mu_t1) - torch.mean(mu_t0)

tensor(10.1807, grad_fn=<SubBackward0>)

In [10]:
print(mu_L.item(), sigma_L.item())

2.4290895462036133 0.2676357924938202


In [12]:
import numpy as np
4*np.exp(1)

10.87312731383618